# Assignment 3.4a & 3.4b

## Pair Members:
Maaz Javaid Siddique (2303.KHI.DEG.004)
Syed Muhammad Raqim Ali Shah (2303.KHI.DEG.008)

Helping hand:  Muhammad Safi (2303.KHI.DEG.023)
Safi helped us in finding bugs in the code. 



In [1]:
!python3 -c "import sys; print(sys.executable)"
!echo $CONDA_EXE


/home/maazjavaidsiddique/Desktop/Assignment/mlops-student/bin/python3
/home/maazjavaidsiddique/anaconda3/bin/conda


# MLFlow lab

In [2]:
import pandas as pd


In [3]:
pd.__version__

'1.5.3'

### Setting up MLFlow tracking server

We also specify artifact root and backend store URI. This makes it possible to store models.

After running this command tracking server will be accessible at `localhost:5000`

In [4]:
%%bash --bg

mlflow server --host 0.0.0.0 \
    --port 5000 \
    --backend-store-uri sqlite:///mlflow.db \
    --default-artifact-root ./mlruns

### MLProject file

This file is used to configure MLFlow steps.

Using `MLproject` we can define our project's pipeline steps, called *entry points*.

Each entry point in this file corresponds to a shell command.

Entry points can be ran using

```
mlflow run -e <ENTRY_POINT>
```

By default `mlflow run` runs `main` entrypoint.

In [24]:
%cat MLproject

name: basic_mlflow

# this file is used to configure Python package dependencies.
# it uses Anaconda, but it can be also alternatively configured to use pip.
conda_env: conda.yaml

# entry points can be ran using `mlflow run <project_name> -e <entry_point_name>
entry_points:
  download_data:
    # you can run any command using MLFlow
    command: "bash download_data.sh"
  # MLproject file has to have main entry_point. It can be toggled without using -e option.
  main:
    # parameters is a key-value collection.
    parameters:
      file_name:
        type: str
        default: "wine_dataset.csv"
      max_k:
        type: int
        default: 10
    command: "python train.py {file_name} {max_k}"



First we need to download data. We will use weather data from previous machine learning tutorial.

## Training

Now we can train models. See `train.py`.
It contains code from supervised machine learning tutorial; we added tracking metrics and model.

We will train kNN models for $k \in \{1, 2, ..., 10\}$ using *temperature* and *casual* features.

After running this command you can go to `localhost:5000` and see the trained models.

In [6]:
import sklearn


In [7]:
sklearn.__version__


'1.1.3'

In [8]:
%%bash
# Source environment variables for MLflow configuration.
source mlflow_env_vars.sh

# Run the MLflow project located in the current directory.
mlflow run .

2023/05/08 08:48:46 INFO mlflow.utils.conda: Conda environment mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33 already exists.
2023/05/08 08:48:46 INFO mlflow.projects.utils: === Created directory /tmp/tmp91i8wray for downloading remote URIs passed to arguments of type 'path' ===
2023/05/08 08:48:46 INFO mlflow.projects.backend.local: === Running command 'source /home/maazjavaidsiddique/anaconda3/bin/../etc/profile.d/conda.sh && conda activate mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33 1>&2 && python train.py day.csv 10' in run with ID '2dc1c43c43f546729b38fdceabf10ac2' === 
/home/maazjavaidsiddique/Desktop/Assignment/mlops-student/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'LogReg' already exists. Creating a new version of this model...
2023/05/08 08:48:48 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version t

## Inspecting stored models

The trained models are stored in `mlruns/0`.

These directories contain artifacts and config that is needed to serve them.

In [23]:
%%bash
last_model_path=$(ls -tr mlruns/0/ | tail -1)
cat mlruns/0/$last_model_path/artifacts/LogisticRegression/MLmodel

artifact_path: LogisticRegression
flavors:
  python_function:
    env:
      conda: conda.yaml
      virtualenv: python_env.yaml
    loader_module: mlflow.sklearn
    model_path: model.pkl
    predict_fn: predict
    python_version: 3.10.6
  sklearn:
    code: null
    pickled_model: model.pkl
    serialization_format: cloudpickle
    sklearn_version: 1.2.2
mlflow_version: 2.3.1
model_uuid: acd7871e063d4534bdf6b91f4054610c
run_id: cae9bcb12c424e49a81cfcb5be45cc78
utc_time_created: '2023-05-08 03:48:56.127711'


In [16]:
import mlflow

In [17]:
mlflow.__version__

'2.3.1'

## Serving model

Now that we trained our models we can go to *Models* page on MLFLow UI (http://localhost:5000/#/models).

Click *sklearn_knn* on this page, choose a model and move it to *Production* stage.

The following cell will serve the model at localhost on port 5001.

In [18]:
%%bash --bg
source mlflow_env_vars.sh
mlflow --version
mlflow models serve -m models:/LogReg/Production -p 5001 --env-manager=conda 


# Prediction

We'll load data that we can feed into prediction server.

In [19]:
from sklearn import datasets

In [20]:
wine=datasets.load_wine()
wine_df=pd.DataFrame(data=wine.data, columns=wine.feature_names)
wine_df.head()


alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0    14.23        1.71  2.43               15.6      127.0           2.80   
1    13.20        1.78  2.14               11.2      100.0           2.65   
2    13.16        2.36  2.67               18.6      101.0           2.80   
3    14.37        1.95  2.50               16.8      113.0           3.85   
4    13.24        2.59  2.87               21.0      118.0           2.80   

   flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
0        3.06                  0.28             2.29             5.64  1.04   
1        2.76                  0.26             1.28             4.38  1.05   
2        3.24                  0.30             2.81             5.68  1.03   
3        3.49                  0.24             2.18             7.80  0.86   
4        2.69                  0.39             1.82             4.32  1.04   

   od280/od315_of_diluted_wines  proline  
0                          3.92   1065.0  
1                          3.40   1050.0  
2                          3.17   1185.0  
3                          3.45   1480.0  
4                          2.93    735.0

Let's predict for first winter day and first non-winter day (first rows of previous two dataframes)

**warning: this might fail at first because the prediction server didn't spin up; in this case wait a minute**

In [21]:
%%bash
data='[[14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0]]'
echo $data

curl -d "{\"inputs\": $data}" -H 'Content-Type: application/json' 127.0.0.1:5001/invocations

[[14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   104  100    20  100    84   7399  31076 --:--:-- --:--:-- --:--:-- 52000


{"predictions": [0]}

In [22]:
%%bash
data='[[13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0]]'
echo $data

curl -d "{\"instances\": $data}" -H 'Content-Type: application/json' 127.0.0.1:5001/invocations

[[13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   107  100    20  100    87   7695  33474 --:--:-- --:--:-- --:--:-- 53500


{"predictions": [0]}

Voila! We see that the model outputs correct predictions.